**Importing the needed packages**

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from scipy.stats import pearsonr
from deap import base, creator, tools, algorithms
import lightgbm as lgb
import random
from sklearn.preprocessing import LabelEncoder

In [8]:
df_monday = pd.read_csv("C:/VS code projects/data_files/Monday-WorkingHours.pcap_ISCX.csv")
df_tuesday = pd.read_csv("C:/VS code projects/data_files/Tuesday-WorkingHours.pcap_ISCX.csv")
df_wednesday = pd.read_csv("C:/VS code projects/data_files/Wednesday-workingHours.pcap_ISCX.csv")
df_thursday_afternoon = pd.read_csv("C:/VS code projects/data_files/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
df_thursday_morning= pd.read_csv("C:/VS code projects/data_files/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
df_friday_afternoon_ddos = pd.read_csv("C:/VS code projects/data_files/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df_friday_afternoon_portscan = pd.read_csv("C:/VS code projects/data_files/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df_friday_morning = pd.read_csv("C:/VS code projects/data_files/Friday-WorkingHours-Morning.pcap_ISCX.csv")
df_monday.columns = df_monday.columns.str.strip()
df_tuesday.columns = df_tuesday.columns.str.strip()
df_wednesday.columns = df_wednesday.columns.str.strip()
df_thursday_afternoon.columns = df_thursday_afternoon.columns.str.strip()
df_thursday_morning.columns = df_thursday_morning.columns.str.strip()
df_friday_afternoon_ddos.columns = df_friday_afternoon_ddos.columns.str.strip()
df_friday_afternoon_portscan.columns = df_friday_afternoon_portscan.columns.str.strip()
df_friday_morning.columns = df_friday_morning.columns.str.strip()


In [9]:
df = pd.concat([df_monday, df_tuesday, df_wednesday, df_thursday_afternoon, df_thursday_morning, df_friday_afternoon_ddos, df_friday_afternoon_portscan, df_friday_morning], ignore_index=True)

In [10]:
# Preprocessing dataset
print("Preprocessing dataset...")

# Step 1: Replace infinite values with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Step 2: Handle NaN values (e.g., fill with 0 or column mean)
df.fillna(0, inplace=True)

print("Done preprocessing dataset")

Preprocessing dataset...
Done preprocessing dataset


In [11]:
# Step 3: Encode categorical labels
label_encoder = LabelEncoder()
print(df['Label'].value_counts())
df['Label'] = label_encoder.fit_transform(df['Label'])
print("Labels encoded successfully!")
df['Label'].value_counts()

Label
BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64
Labels encoded successfully!


Label
0     2273097
4      231073
10     158930
2      128027
3       10293
7        7938
11       5897
6        5796
5        5499
1        1966
12       1507
14        652
9          36
13         21
8          11
Name: count, dtype: int64

In [13]:
# Step 4: Separate features and labels
X = df.drop('Label', axis=1)
Y = df['Label']

In [14]:
# Step 5: Normalize numerical features using MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [15]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [16]:
# Filter Method: Calculate correlations between features and labels
print("Calculating feature importance using correlation...")
correlations = []
for i in range(X_train.shape[1]):
    if np.std(X_train[:, i]) == 0:  # Skip constant features
        correlations.append(0)
    else:
        correlations.append(abs(pearsonr(X_train[:, i], y_train)[0]))
correlation_threshold = 0.2  # Define a threshold to filter irrelevant features
relevant_features = [i for i, corr in enumerate(correlations) if corr > correlation_threshold]
print(f"Features with correlation above threshold: {len(relevant_features)}")

# Subset the data with relevant features only
X_train = X_train[:, relevant_features]
X_test = X_test[:, relevant_features]
feature_names = [df.columns[i] for i in relevant_features]
[print(f"{i}: {feature_names[i]}") for i in range(len(feature_names))]

Calculating feature importance using correlation...
Features with correlation above threshold: 4
0: Bwd Packet Length Min
1: Fwd IAT Std
2: Min Packet Length
3: PSH Flag Count


[None, None, None, None]

In [17]:
# GA Feature Selection
print("Defining Genetic Algorithm for feature selection...")

# Define GA components
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=X_train.shape[1])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

Defining Genetic Algorithm for feature selection...


In [19]:
# Define the fitness function
def fitness(individual, X_train, y_train):
    selected_features = [i for i, bit in enumerate(individual) if bit == 1]
    if len(selected_features) == 0:  # Avoid empty feature sets
        return 0,
    X_train_selected = X_train[:, selected_features]
    model = lgb.LGBMClassifier(random_state=42)
    model.fit(X_train_selected, y_train)
    accuracy = model.score(X_train_selected, y_train)  # Training accuracy
    return accuracy,

toolbox.register("evaluate", fitness, X_train=X_train, y_train=y_train)

In [ ]:
# Set GA parameters
population = toolbox.population(n=50)
ngen = 40
cxpb = 0.7
mutpb = 0.2

# Run the GA
result_population = algorithms.eaSimple(population, toolbox, cxpb=cxpb, mutpb=mutpb, ngen=ngen, verbose=False)
best_individual = tools.selBest(result_population[0], k=1)[0]
selected_features = [i for i, bit in enumerate(best_individual) if bit == 1]

# Print the chosen features
chosen_features = [feature_names[i] for i in selected_features]
print("Selected Features:")
print(chosen_features)

In [ ]:
# Train a model on the selected features
print("Training final model with selected features...")
X_train_selected = X_train[:, selected_features]
X_test_selected = X_test[:, selected_features]
X_train_selected = pd.DataFrame(X_train_selected, columns=[feature_names[i] for i in selected_features])
X_test_selected = pd.DataFrame(X_test_selected, columns=[feature_names[i] for i in selected_features])

final_model = lgb.LGBMClassifier(
    random_state=42,
    learning_rate=0.05,
    num_leaves=50,
    max_depth=10,
    n_estimators=200,
    boosting_type='gbdt',
    class_weight='balanced',
    verbose_eval= 1  # Print progress every iteration

)
final_model.fit(X_train_selected, y_train,    
                 eval_set=[(X_test_selected, y_test)],  # Validation set
                 eval_metric='logloss',                # Metric to monitor
                 early_stopping_rounds=20,             # Stop if no improvement in 20 rounds
                 verbose=10   )

Training final model with selected features...
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004314 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 622
[LightGBM] [Info] Number of data points in the train set: 253031, number of used features: 4
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from

LGBMClassifier(class_weight='balanced', learning_rate=0.05, max_depth=10,
               n_estimators=200, num_leaves=50, random_state=42,
               verbose_eval=1)

In [ ]:
# Evaluate the model
y_pred = final_model.predict(X_test_selected)


# Get feature importance
feature_importance = final_model.feature_importances_

# Combine feature names and their importance into a DataFrame for better readability
feature_importance_df = pd.DataFrame({
    'Feature': X_train_selected.columns,
    'Importance': feature_importance
}).sort_values(by='Importance', ascending=False)

[LightGBM] [Warning] Unknown parameter: verbose_eval


In [ ]:
# Print the feature importance
print("\nFeature Importance:")
print(feature_importance_df)


Feature Importance:
                  Feature  Importance
3  Init_Win_bytes_forward       48313
1       Min Packet Length       10591
0   Bwd Packet Length Min       10228
2          PSH Flag Count        1759


In [ ]:
# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
# Print evaluation metrics
print(f"Final Model Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

Final Model Accuracy: 0.8387
Precision: 0.9440
Recall: 0.8387
F1 Score: 0.8745
Confusion Matrix:
[[44837  7179   187   180  5462  1237   663    30]
 [    8 11520     0     2     0     0     0    14]
 [    0     0   269     0   675     0     0     0]
 [    5     0   116 19339  1279     0     0     0]
 [    4     0     0     0   405    44    21     0]
 [    0     0     0     0   321   154     1     0]
 [    0     0     0     0     0     0     1     0]
 [    0     0     0     4    63     0     0 14423]]


In [ ]:
#Print a detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.75      0.86     59775
           1       0.62      1.00      0.76     11544
           2       0.47      0.28      0.35       944
           3       0.99      0.93      0.96     20739
           4       0.05      0.85      0.09       474
           5       0.11      0.32      0.16       476
           6       0.00      1.00      0.00         1
           7       1.00      1.00      1.00     14490

    accuracy                           0.84    108443
   macro avg       0.53      0.77      0.52    108443
weighted avg       0.94      0.84      0.87    108443

